# Step 1. Load Modules

In [78]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Step 2. Use mapping csv to transform lab data.

In [79]:
mapping = pd.read_csv('../data/itemid_to_variable_map.csv', index_col='ITEMID', dtype={'ITEMID': int})
mapping = mapping[(mapping['LEVEL2'] != '') &\
                  (mapping['COUNT'] > 0) &\
                  (mapping['STATUS'] == 'ready')
                 ]

In [80]:
mapping.head()

,LEVEL2,LEVEL1,ALTERNATIVE,STATUS,STATUS NOTE,MIMIC LABEL,UNITNAME,LINKSTO,COUNT,CATEGORY,CONCEPTID,FLUID,LOINC_CODE,DBSOURCE,Unnamed: 15,PARAM_TYPE,NOTE
ITEMID,,,,,,,,,,,,,,,,,
50861,Alanine aminotransferase,Alanine aminotransferase,ALT,ready,NaN,ALANINE AMINOTRANSFERASE (ALT),NaN,labevents,219475.0,CHEMISTRY,NaN,BLOOD,,NaN,NaN,NaN,NaN
769,Alanine aminotransferase,Alanine aminotransferase,ALT,ready,NaN,ALT,NaN,chartevents,41594.0,Enzymes,NaN,NaN,NaN,carevue,NaN,NaN,NaN
220644,Alanine aminotransferase,Alanine aminotransferase,ALT,ready,NaN,ALT,None,chartevents,37625.0,Labs,NaN,NaN,NaN,metavision,NaN,Numeric,NaN
50862,Albumin,Albumin,NaN,ready,NaN,ALBUMIN,NaN,labevents,146697.0,CHEMISTRY,NaN,BLOOD,1751-7,NaN,NaN,NaN,NaN
772,Albumin,Albumin,NaN,ready,NaN,Albumin (>3.2),NaN,chartevents,31022.0,Chemistry,NaN,NaN,NaN,carevue,NaN,NaN,NaN


In [81]:
lab_data = pd.read_csv('../data/LABEVENTS.csv', parse_dates=['CHARTTIME'])
lab_data = lab_data[lab_data['ITEMID'].isin(mapping.index)][['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM']]
lab_data['Lab'] = mapping['LEVEL1'].loc[lab_data['ITEMID']].values

In [82]:
lab_data.head()

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUENUM,Lab
0,3,NaN,50820,2101-10-12 16:07:00,7.39,pH
3,3,NaN,50804,2101-10-12 18:17:00,22.00,"CO2 (ETCO2, PCO2, etc.)"
4,3,NaN,50808,2101-10-12 18:17:00,0.93,Calcium ionized
6,3,NaN,50813,2101-10-12 18:17:00,1.80,Lactate
7,3,NaN,50818,2101-10-12 18:17:00,33.00,Partial pressure of carbon dioxide


# Step 3. Load in Admissions csv and merge with DOB/Gender information.

In [83]:
patients = pd.read_csv('../data/PATIENTS.csv', usecols = ['SUBJECT_ID', 'GENDER', 'DOB'], parse_dates=['DOB'])
patients.head()

,SUBJECT_ID,GENDER,DOB
0,249,F,2075-03-13
1,250,F,2164-12-27
2,251,M,2090-03-15
3,252,M,2078-03-06
4,253,F,2089-11-26


In [84]:
admissions = pd.read_csv('../data/ADMISSIONS.csv',
                         usecols = ['SUBJECT_ID', 'HADM_ID', 'ADMISSION_TYPE', 'HOSPITAL_EXPIRE_FLAG',
                                    'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ETHNICITY', 'INSURANCE', 'DIAGNOSIS'],
                         parse_dates=['ADMITTIME', 'DISCHTIME', 'DEATHTIME'])

In [85]:
admissions_merged = admissions.merge(patients, on='SUBJECT_ID')

# Step 4. Filter only for adults in admissions_merged.

In [86]:
admissions = admissions_merged.dropna(subset=['ADMITTIME', 'DOB'])

admissions.shape

(58976, 12)

In [87]:
admissions['AGE'] = admissions['ADMITTIME'].dt.to_pydatetime() - admissions['DOB'].dt.to_pydatetime()
admissions['AGE'] = [date.days/365 for date in admissions['AGE']]
admissions_filtered = admissions[admissions['AGE'] >= 18]

admissions_filtered.shape

(50766, 13)

# Step 5. Clean admissions.

In [88]:
# Get last visit for each patient.
admissions = admissions_filtered.loc[admissions_filtered['SUBJECT_ID'].drop_duplicates('last').index]

# Change times to hours since admissions
admissions['Death'] = admissions['DEATHTIME'] - admissions['ADMITTIME']
admissions['LOS'] = admissions['DISCHTIME'] - admissions['ADMITTIME']

# Create index for easier search
admissions = admissions.set_index('SUBJECT_ID')
admissions.shape

C:\Users\steph\AppData\Local\Temp\ipykernel_18448\942370027.py:2: FutureWarning: In a future version of pandas all arguments of Series.drop_duplicates will be keyword-only.
  admissions = admissions_filtered.loc[admissions_filtered['SUBJECT_ID'].drop_duplicates('last').index]


(38552, 14)

In [89]:
assert len(admissions.HADM_ID.unique()) == len(admissions), \
    'Different patients have the same HADM_ID'

# Step 6. Clean lab_data

In [90]:
lab_data = lab_data[lab_data['HADM_ID'].isin(admissions['HADM_ID'])]

# like admissions time transformations, change time event to time since admission
lab_data['Time'] = lab_data['CHARTTIME'].values - admissions['ADMITTIME'].loc[lab_data['SUBJECT_ID']].values

lab_data.head()

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUENUM,Lab,Time
160,3,145834.0,50868,2101-10-20 16:40:00,17.0,Anion gap,-1 days +21:32:00
161,3,145834.0,50882,2101-10-20 16:40:00,25.0,Bicarbonate,-1 days +21:32:00
162,3,145834.0,50893,2101-10-20 16:40:00,8.2,Calcium (total),-1 days +21:32:00
163,3,145834.0,50902,2101-10-20 16:40:00,99.0,Chloride,-1 days +21:32:00
166,3,145834.0,50912,2101-10-20 16:40:00,3.2,Creatinine,-1 days +21:32:00


# Step 7. Reformat dataframes for csv output

In [91]:
lab_data.rename(columns={'SUBJECT_ID': 'Patient', 'VALUENUM': 'Value'}, inplace = True)
lab_data = lab_data.reset_index(drop = True)[['Patient', 'Time', 'Lab', 'Value']]
lab_data.head()

,Patient,Time,Lab,Value
0,3,-1 days +21:32:00,Anion gap,17.0
1,3,-1 days +21:32:00,Bicarbonate,25.0
2,3,-1 days +21:32:00,Calcium (total),8.2
3,3,-1 days +21:32:00,Chloride,99.0
4,3,-1 days +21:32:00,Creatinine,3.2


In [92]:
admissions.rename_axis(index = 'Patient', inplace = True)
admissions.head()

,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,INSURANCE,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,DOB,AGE,Death,LOS
Patient,,,,,,,,,,,,,,
22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaT,EMERGENCY,Private,WHITE,BENZODIAZEPINE OVERDOSE,0,F,2131-05-07,64.969863,NaT,1 days 03:28:00
23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,EMERGENCY,Medicare,WHITE,BRAIN MASS,0,M,2082-07-17,75.304110,NaT,6 days 18:26:00
24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaT,EMERGENCY,Private,WHITE,INTERIOR MYOCARDIAL INFARCTION,0,M,2100-05-31,39.041096,NaT,2 days 20:34:00
25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaT,EMERGENCY,Private,WHITE,ACUTE CORONARY SYNDROME,0,M,2101-11-21,58.989041,NaT,3 days 12:49:00
26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaT,EMERGENCY,Medicare,UNKNOWN/NOT SPECIFIED,V-TACH,0,M,2054-05-04,72.052055,NaT,6 days 23:44:00


# Step 8. Filter results for visits of length 24 hours or more.

In [93]:
time_threshold = pd.to_timedelta('1 day'.format(1))
admissions = admissions[admissions['LOS'] >= time_threshold]
lab_data = lab_data[(lab_data['Time'] < time_threshold) & (lab_data['Time'] > pd.to_timedelta('0 day')) & lab_data['Patient'].isin(admissions.index)]

# Step 9. Clean lab_data

In [94]:
# remove duplicates
lab_data = lab_data[~lab_data.set_index(['Patient', 'Time', 'Lab']).index.duplicated(keep=False)]

# pivot to make tests columns
lab_data = lab_data.pivot(index=['Patient', 'Time'], columns = 'Lab')

# change index to have days
lab_data.index = lab_data.index.set_levels(lab_data.index.levels[1].total_seconds()/(3600.*24), 1)

#lab_data = lab_data.reset_index()

# remove empty lines
lab_data = lab_data.dropna(how='all')

lab_data.head()

C:\Users\steph\AppData\Local\Temp\ipykernel_18448\2852689128.py:8: FutureWarning: In a future version of pandas all arguments of MultiIndex.set_levels except for the argument 'levels' will be keyword-only.
  lab_data.index = lab_data.index.set_levels(lab_data.index.levels[1].total_seconds()/(3600.*24), 1)


Value                            \
Lab              Alanine aminotransferase Albumin Albumin (ascites)   
Patient Time                                                          
3       0.002778                      NaN     NaN               NaN   
        0.004167                      NaN     NaN               NaN   
        0.012500                      NaN     NaN               NaN   
        0.035417                     25.0     1.8               NaN   
        0.038889                      NaN     NaN               NaN   

                                                                       \
Lab              Albumin (pleural) Albumin (urine) Alkaline phosphate   
Patient Time                                                            
3       0.002778               NaN             NaN                NaN   
        0.004167               NaN             NaN                NaN   
        0.012500               NaN             NaN                NaN   
        0.035417               NaN             NaN               73.0   
        0.038889               NaN             NaN                NaN   

                                                                            \
Lab              Anion gap Asparate aminotransferase Basophils Bicarbonate   
Patient Time                                                                 
3       0.002778       NaN                       NaN       NaN         NaN   
        0.004167       NaN                       NaN       NaN         NaN   
        0.012500      23.0                       NaN       NaN        13.0   
        0.035417      22.0                      69.0       NaN        11.0   
        0.038889       NaN                       NaN       NaN         NaN   

                  ...                                 \
Lab               ... Red blood cell count (ascites)   
Patient Time      ...                                  
3       0.002778  ...                            NaN   
        0.004167  ...                            NaN   
        0.012500  ...                            NaN   
        0.035417  ...                            NaN   
        0.038889  ...                            NaN   

                                                                             \
Lab              Red blood cell count (pleural) Sodium Sodium (whole blood)   
Patient Time                                                                  
3       0.002778                            NaN    NaN                138.0   
        0.004167                            NaN    NaN                153.0   
        0.012500                            NaN  143.0                  NaN   
        0.035417                            NaN  140.0                  NaN   
        0.038889                            NaN    NaN                  NaN   

                                                                       \
Lab              Tidal Volume Observed Total Protein Urine Troponin-I   
Patient Time                                                            
3       0.002778                   NaN                 NaN        NaN   
        0.004167                   NaN                 NaN        NaN   
        0.012500                   NaN                 NaN        NaN   
        0.035417                   NaN                 NaN        NaN   
        0.038889                   NaN                 NaN        NaN   

                                                          
Lab              Troponin-T White blood cell count    pH  
Patient Time                                              
3       0.002778        NaN                    NaN  7.35  
        0.004167        NaN                    NaN  7.59  
        0.012500        NaN                   11.3   NaN  
        0.035417        NaN                   19.1   NaN  
        0.038889        NaN                    NaN  7.26  

[5 rows x 67 columns]

# Step 10. Clean admissions

In [95]:
# remove patients with no lab data
admissions = admissions.loc[lab_data.index.get_level_values(0).unique()]

admissions['Death'] = admissions['Death'].dt.total_seconds() / (24 * 60 * 60)

In [96]:
admissions.shape

(36296, 14)

In [97]:
lab_data.shape

(211550, 67)

In [104]:
new_lab_data.reset_index()
new_lab.head()

Patient      Time                    Value                            \
Lab                   Alanine aminotransferase Albumin Albumin (ascites)   
0         3  0.002778                      NaN     NaN               NaN   
1         3  0.004167                      NaN     NaN               NaN   
2         3  0.012500                      NaN     NaN               NaN   
3         3  0.035417                     25.0     1.8               NaN   
4         3  0.038889                      NaN     NaN               NaN   

                                                                    \
Lab Albumin (pleural) Albumin (urine) Alkaline phosphate Anion gap   
0                 NaN             NaN                NaN       NaN   
1                 NaN             NaN                NaN       NaN   
2                 NaN             NaN                NaN      23.0   
3                 NaN             NaN               73.0      22.0   
4                 NaN             NaN                NaN       NaN   

                               ...                                 \
Lab Asparate aminotransferase  ... Red blood cell count (ascites)   
0                         NaN  ...                            NaN   
1                         NaN  ...                            NaN   
2                         NaN  ...                            NaN   
3                        69.0  ...                            NaN   
4                         NaN  ...                            NaN   

                                                                \
Lab Red blood cell count (pleural) Sodium Sodium (whole blood)   
0                              NaN    NaN                138.0   
1                              NaN    NaN                153.0   
2                              NaN  143.0                  NaN   
3                              NaN  140.0                  NaN   
4                              NaN    NaN                  NaN   

                                                                     \
Lab Tidal Volume Observed Total Protein Urine Troponin-I Troponin-T   
0                     NaN                 NaN        NaN        NaN   
1                     NaN                 NaN        NaN        NaN   
2                     NaN                 NaN        NaN        NaN   
3                     NaN                 NaN        NaN        NaN   
4                     NaN                 NaN        NaN        NaN   

                                  
Lab White blood cell count    pH  
0                      NaN  7.35  
1                      NaN  7.59  
2                     11.3   NaN  
3                     19.1   NaN  
4                      NaN  7.26  

[5 rows x 69 columns]

# Step 10. Save dataframes to csv format

In [105]:
new_lab_data.to_csv('../data/lab_data_cleaned.csv')
admissions.to_csv('../data/admissions_cleaned.csv')